In [275]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import urllib, json
from PIL import Image
import datetime

In [276]:
# Подключаем api_key из файла
f = open('API_KEY.txt', 'r')
api_key = f.read()
f.close()

In [277]:
# Составляем URL request
api_root = 'https://api.nasa.gov/'
api_product = 'neo/rest/v1/feed?'
start_date = '2020-07-07'
end_date = '2020-07-07'
url = f"{api_root}{api_product}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [278]:
'''
    Функция навигации по данным из api response
    возвращает ссылку на данные в другом временном отрезке

    step = 'next' -- следующий день от заданного start_date в URL request
    step = 'prev' -- предыдущий день от заданного start_date в URL request
    step = 'self' -- текущий день, равный start_date в URL request
'''

def get_url(url, step):
    url_level = urllib.request.urlopen(url)
    data_level = json.loads(url_level.read())['links']
    if step == 'next':
        return data_level[step]
    elif step == 'prev':
        return data_level[step]
    elif step == 'self':
        return data_level[step]

In [279]:
# Навигация по ссылкам
# Работаем с данными за один день 2020-07-07
url_level_0 = urllib.request.urlopen(url)
data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
data_level_0

,links,element_count,near_earth_objects
next,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
prev,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
self,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
2020-07-07,NaN,11,[{'links': {'self': 'http://api.nasa.gov/neo/r...


In [280]:
# Выбираем один элемент из списка за один день
data_day = data_level_0.near_earth_objects[3]
data_day_item = data_day[0]
data_day_item

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3343102?api_key=8Mbg6rxSagXNYxrUbC8ynidJUcLn9h4CC7ucuT21'},
 'id': '3343102',
 'neo_reference_id': '3343102',
 'name': '(2006 SS19)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3343102',
 'absolute_magnitude_h': 18.9,
 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.4411182,
   'estimated_diameter_max': 0.9863702813},
  'meters': {'estimated_diameter_min': 441.1181999969,
   'estimated_diameter_max': 986.3702813054},
  'miles': {'estimated_diameter_min': 0.2740980571,
   'estimated_diameter_max': 0.6129018881},
  'feet': {'estimated_diameter_min': 1447.2382352778,
   'estimated_diameter_max': 3236.1230737181}},
 'is_potentially_hazardous_asteroid': False,
 'close_approach_data': [{'close_approach_date': '2020-07-07',
   'close_approach_date_full': '2020-Jul-07 08:13',
   'epoch_date_close_approach': 1594109580000,
   'relative_velocity': {'kilometers_per_second': '11.947301649',
    'kilometers_per_h

In [281]:
# Создаем словари с выбранными данными для DataFrame
dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
dict_close_approach_data = data_day_item['close_approach_data']
dict_relative_velocity = dict_close_approach_data[0]['relative_velocity']
dict_miss_distance = dict_close_approach_data[0]['miss_distance']

In [282]:
# Создаем DataFrame
# Объединяем DataFrame
df1 = pd.DataFrame(data_day_item, index=['1'])
df2 = pd.DataFrame(dict_estimated_diameter, index=['1'])
df3 = pd.DataFrame(dict_close_approach_data, index=['1'])
df4 = pd.DataFrame(dict_relative_velocity, index=['1'])
df5 = pd.DataFrame(dict_miss_distance, index=['1'])
df1.join(df2).join(df3).join(df4).join(df5)

,links,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,estimated_diameter,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,relative_velocity,miss_distance,orbiting_body,kilometers_per_second,kilometers_per_hour,miles_per_hour,astronomical,lunar,kilometers,miles
1,NaN,3343102,3343102,(2006 SS19),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3343102,18.9,NaN,False,"{'close_approach_date': '2020-07-07', 'close_a...",False,...,"{'kilometers_per_second': '11.947301649', 'kil...","{'astronomical': '0.4460874678', 'lunar': '173...",Earth,11.947301649,43010.285936445,26724.9190586761,0.4460874678,173.5280249742,66733735.016573586,41466420.1158480468


In [283]:
# Чистим DataFrame от нерелевантных данных, переименовываем
df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['links',
                                                  'nasa_jpl_url',
                                                  'neo_reference_id',
                                                  'estimated_diameter',
                                                  'close_approach_data',
                                                  'relative_velocity',
                                                  'miss_distance',
                                                  'miles_per_hour',
                                                  'astronomical',
                                                  'lunar',
                                                  'miles'
                                                  ], axis=1
).rename(columns={'kilometers': 'miss_distance_kilometers',
                  'estimated_diameter_min': 'estimated_diameter_km_min',
                  'estimated_diameter_max': 'estimated_diameter_km_max'})

df_item

,id,name,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_km_min,estimated_diameter_km_max,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,kilometers_per_second,kilometers_per_hour,miss_distance_kilometers
1,3343102,(2006 SS19),18.9,False,False,0.441118,0.98637,2020-07-07,2020-Jul-07 08:13,1594109580000,Earth,11.947301649,43010.285936445,66733735.016573586
